In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from t5.data import preprocessors as prep
import functools
import t5
import gin
import sentencepiece as spm
from glob import glob
import os

gin.parse_config_file('pretrained_models_base_operative_config.gin')
vocab = 'sp10m.cased.ms-en.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)

True

In [5]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/corpus/goemotions/goemotions_1.translated.csv
# !wget https://f000.backblazeb2.com/file/malay-dataset/corpus/goemotions/goemotions_2.translated.csv
# !wget https://f000.backblazeb2.com/file/malay-dataset/corpus/goemotions/goemotions_3.translated.csv

['goemotions_3.translated.csv',
 'goemotions_2.translated.csv',
 'goemotions_1.translated.csv']

In [6]:
import pandas as pd

df = pd.read_csv('goemotions_1.translated.csv')
df.head()

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,translate_nmt,translate_replace
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,...,0,0,0,0,0,1,0,0,Permainan itu sakit.,That permainan sakit hati .
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1.548084e+09,37,True,0,...,0,0,0,0,0,0,0,0,> seksualiti tidak boleh menjadi kategori peng...,> seksualiti sepatutnya tidak jadi a kumpulan ...
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1.546428e+09,37,False,0,...,0,0,0,0,0,0,0,1,"Anda melakukan yang betul, jika anda tidak ped...","You do right , jika awak do tidak jaga kemudia..."
3,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1.547965e+09,18,False,0,...,0,0,0,0,0,0,0,0,Lelaki yang saya suka reddit.,Man I love reddit .
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1.546669e+09,2,False,0,...,0,0,0,0,0,0,0,1,[NAME] tidak ada di mana-mana berhampiran mere...,[ NAME ] adalah tiada tempat berhampiran them ...


In [15]:
from tqdm import tqdm

en_ms_translated = []
en_ms_replaced = []
ms_en_translated = []
ms_en_replaced = []

for f in glob('goemotions_*.translated.csv'):
    print(f)
    df = pd.read_csv(f)

    for i in tqdm(range(len(df))):
        en_ms_translated.append((df.iloc[i,0], df.iloc[i,-2]))
        en_ms_replaced.append((df.iloc[i,0], df.iloc[i,-1]))
        ms_en_translated.append((df.iloc[i,-2], df.iloc[i,0]))
        ms_en_replaced.append((df.iloc[i,-1], df.iloc[i,0]))

goemotions_3.translated.csv


100%|██████████| 71225/71225 [00:19<00:00, 3600.23it/s]


goemotions_2.translated.csv


100%|██████████| 70000/70000 [00:19<00:00, 3607.38it/s]


goemotions_1.translated.csv


100%|██████████| 70000/70000 [00:18<00:00, 3703.47it/s]


In [16]:
import re

def cleaning(string):
    string = string.replace('\n', ' ').replace('\t', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

In [20]:
with tf.io.gfile.GFile('en_ms_translated.tsv', "w") as outfile:
    for i in tqdm(range(len(en_ms_translated))):
        try:
            l = cleaning(en_ms_translated[i][0])
            r = cleaning(en_ms_translated[i][1])
            outfile.write("%s\t%s\n" % (l, r))
        except:
            pass

100%|██████████| 211225/211225 [00:04<00:00, 48736.72it/s]


In [22]:
with tf.io.gfile.GFile('en_ms_replaced.tsv', "w") as outfile:
    for i in tqdm(range(len(en_ms_replaced))):
        try:
            l = cleaning(en_ms_replaced[i][0])
            r = cleaning(en_ms_replaced[i][1])
            outfile.write("%s\t%s\n" % (l, r))
        except:
            pass

100%|██████████| 211225/211225 [00:04<00:00, 44135.35it/s]


In [23]:
with tf.io.gfile.GFile('ms_en_translated.tsv', "w") as outfile:
    for i in tqdm(range(len(ms_en_translated))):
        try:
            l = cleaning(ms_en_translated[i][0])
            r = cleaning(ms_en_translated[i][1])
            outfile.write("%s\t%s\n" % (l, r))
        except:
            pass

100%|██████████| 211225/211225 [00:04<00:00, 50799.15it/s]


In [24]:
with tf.io.gfile.GFile('ms_en_replaced.tsv', "w") as outfile:
    for i in tqdm(range(len(ms_en_replaced))):
        try:
            l = cleaning(ms_en_replaced[i][0])
            r = cleaning(ms_en_replaced[i][1])
            outfile.write("%s\t%s\n" % (l, r))
        except:
            pass

100%|██████████| 211225/211225 [00:04<00:00, 46304.85it/s]


In [33]:
def social_en_ms_translated_dataset(split, shuffle_files = False):
    del shuffle_files
    ds = tf.data.TextLineDataset(
        [
            'ms_en_replaced.tsv'
        ]
    )

    ds = ds.map(
        functools.partial(
            tf.io.decode_csv,
            record_defaults = ['', ''],
            field_delim = '\t',
            use_quote_delim = False,
        ),
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )
    ds = ds.map(lambda *ex: dict(zip(['question', 'answer'], ex)))
    return ds

def social_en_ms_translated_preprocessor(ds):
    def to_inputs_and_targets(ex):
        return {
            'inputs': tf.strings.join(['sosial en_ms_translated: ', ex['question']]),
            'targets': ex['answer'],
        }

    return ds.map(
        to_inputs_and_targets,
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )

In [34]:
t5.data.TaskRegistry.remove('social_en_ms_translated_dataset')
t5.data.TaskRegistry.add(
    'social_en_ms_translated_dataset',
    dataset_fn = social_en_ms_translated_dataset,
    splits = ['train'],
    text_preprocessor = [social_en_ms_translated_preprocessor],
    sentencepiece_model_path = vocab,
    metric_fns = [t5.evaluation.metrics.accuracy],
)


In [35]:
nq_task = t5.data.TaskRegistry.get("social_en_ms_translated_dataset")
ds = nq_task.get_dataset(split='paraphrase.tsv', sequence_length={"inputs": 1024, "targets": 1024})
r = tfds.as_numpy(ds)

In [36]:
next(r)

{'inputs_plaintext': b'sosial en_ms_translated: I do tidak ingat [ NAME ] saying itu tbh',
 'inputs': array([  969,    13,   258,  2902,   117,    16,  2902, 12262,  5635,
           72,    31,    59,   184,    30,  4148,   356,   517, 20414,
           13,   332,  1238,    37,    13,    41,   380,   261,     1]),
 'targets_plaintext': b'I don\xe2\x80\x99t remember [NAME] saying that tbh',
 'targets': array([   59,   367,     2,    41,  3548,   356,   584, 20414,   332,
         1238,    27,    13,    41,   380,   261,     1])}